# 盘中动态:20241111

In [1]:
import nbimporter
from 盘前作业20241111 import plotStock

repday='20241111'

定时器

## 1. 今日板块热点


![今日涨幅国泰君安APP](gtja.png)

定时任务运行器：集合竞价和开盘前30分钟

In [2]:
import akshare as ak
import os
symbol='600839'

# 获取股票数据
df = ak.stock_bid_ask_em(symbol=symbol)

# 转换为字典
data_dict = df.set_index('item').to_dict()['value']

In [3]:
data_dict

{'sell_5': 12.35,
 'sell_5_vol': 168300.0,
 'sell_4': 12.34,
 'sell_4_vol': 43100.0,
 'sell_3': 12.33,
 'sell_3_vol': 33400.0,
 'sell_2': 12.32,
 'sell_2_vol': 67400.0,
 'sell_1': 12.31,
 'sell_1_vol': 500.0,
 'buy_1': 12.3,
 'buy_1_vol': 176500.0,
 'buy_2': 12.29,
 'buy_2_vol': 59400.0,
 'buy_3': 12.28,
 'buy_3_vol': 15300.0,
 'buy_4': 12.27,
 'buy_4_vol': 43700.0,
 'buy_5': 12.26,
 'buy_5_vol': 61000.0,
 '最新': 12.3,
 '均价': 12.18,
 '涨幅': -7.03,
 '涨跌': -0.93,
 '总手': 2710536.0,
 '金额': 3301167251.0,
 '换手': 5.87,
 '量比': 3.78,
 '最高': 12.89,
 '最低': 11.91,
 '今开': 12.89,
 '昨收': 13.23,
 '涨停': 14.55,
 '跌停': 11.91,
 '外盘': 1216381.0,
 '内盘': 1494154.0}

In [4]:
import datetime

# 股票代码列表
symbols = ["stock_code1", "stock_code2", "stock_code3"]  # 替换为实际股票代码

def is_within_trading_hours(current_time):
    # 定义交易时间段
    morning_start = current_time.replace(hour=9, minute=15, second=0, microsecond=0)
    morning_end = current_time.replace(hour=11, minute=30, second=0, microsecond=0)
    afternoon_start = current_time.replace(hour=13, minute=0, second=0, microsecond=0)
    afternoon_end = current_time.replace(hour=15, minute=0, microsecond=0)
    
    return (morning_start <= current_time <= morning_end) or (afternoon_start <= current_time <= afternoon_end)

def fetch_and_store(symbol):
    # 获取当前时间
    current_time = datetime.now()
    print(current_time)

    if is_within_trading_hours(current_time):
        # 获取股票数据
        df = ak.stock_bid_ask_em(symbol=symbol)

        # 转换为字典
        data_dict = df.set_index('item').to_dict()['value']

        # 添加时间戳和股票代码到数据字典
        data_dict['timestamp'] = current_time
        data_dict['sym'] = symbol

        # 转换数据为 DataFrame
        transformed_df = pd.DataFrame([data_dict])

        # 将数据插入到 DolphinDB
        s.upload({"data": transformed_df})
        s.run("""
        append!(loadTable("dfs://Quote", "t"), data)
        """)

while True:
    # 使用多线程获取多个股票代码的数据
    with ThreadPoolExecutor(max_workers=len(symbols)) as executor:
        executor.map(fetch_and_store, symbols)

    # 每3秒检查一次
    time.sleep(3)

NameError: name 'ThreadPoolExecutor' is not defined

#### 2.2.1 集合竞价概念板块排名

In [17]:
import datetime
import pandas as pd

#记录排名
# basename: 板块类型（概念板块、行业板块）
# start_time, end_time: 排名时间段
# sector: 板块名称
def recordRank(basename,start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'{basename}{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        if basename=='conceptRank':
            rank=ak.stock_board_concept_name_em()
        elif basename=='industryRank':
            rank=ak.stock_board_industry_name_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('非指定时间段，没有数据源.')
            rank=pd.DataFrame()
    if not rank.empty:
        mvtotal=rank['总市值'].sum()
        rank['市值比']=rank['总市值']/mvtotal
        rank['上涨家数%']=rank['上涨家数']/(rank['上涨家数']+rank['下跌家数'])
        columns=['排名','板块名称','板块代码','涨跌幅','市值比','换手率','上涨家数%','领涨股票','领涨股票-涨跌幅']
        rank=rank[columns]
    return rank

#排名靠前的板块的股票明细排名
# sectorRankDf: 板块排名DataFrame(recordRank生成)
# sectorNum,stockNum: 返回板块数、股票数
def top5sectorStocks(basename,start_time,end_time,sectorRankDf,sectorNum=5,stockNum=5):
    top5={}
    now = datetime.datetime.now()
    tStr = start_time.strftime("%Y%m%d%H%M")
    if sectorRankDf.empty:
        print('非指定时间段，没有数据源。')
    else:
        print('排名前5领涨板块成分股列表。')
        for sym in sectorRankDf['板块名称'].to_list()[:5]:
            csvfile=f'{basename}_{sym}{tStr}.txt'
            if now.time() < end_time and now.time() >= start_time:
                if basename=='conceptRank':
                    top5[sym]=ak.stock_board_concept_cons_em(sym)
                else:
                    top5[sym]=ak.stock_board_industry_cons_em(sym)
                top5[sym].to_csv(csvfile)
            else:
                if os.path.exists(csvfile):
                    top5[sym]=pd.read_csv(csvfile)
                else:
                    print('No record stored and not available from source.')
                    top5[sym]=pd.DataFrame()

            if not top5[sym].empty:
                atotal=top5[sym]['成交额'].sum()
                top5[sym]['成交额比']=top5[sym]['成交额']/atotal
                columns=['代码','名称','涨跌幅','成交额','换手率','市盈率-动态','市净率']
                top5[sym]=top5[sym][columns][:stockNum]
                print(f'板块：{sym}:\n{top5[sym]}\n\n')
    return top5   

In [18]:
#当天集合竞价阶段概念板块排名
basename='conceptRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank0=recordRank(basename,start_time,end_time)
rank0.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,昨日涨停_含一字,BK1050,5.36,0.001210,0.0,0.896552,科创新材,20.27
1,2,昨日涨停,BK0815,4.99,0.001051,0.0,0.876033,科创新材,20.27
2,3,昨日连板_含一字,BK1051,4.27,0.000385,0.0,0.850000,西部创业,10.06
3,4,高带宽内存,BK1152,3.98,0.000089,0.0,1.000000,国芯科技,20.01
4,5,昨日连板,BK0816,3.71,0.000260,0.0,0.837838,动力新科,10.04
5,6,Chiplet概念,BK1101,2.99,0.000366,0.0,0.880000,润欣科技,20.01
6,7,华为欧拉,BK1013,2.88,0.000230,0.0,0.733333,东方通,15.00
7,8,3D玻璃,BK0881,2.13,0.000180,0.0,0.647059,福能东方,19.97
8,9,EDA概念,BK0946,2.12,0.000169,0.0,0.846154,华大九天,12.34
9,10,北斗导航,BK0629,1.93,0.000655,0.0,0.808824,杰创智能,14.46


In [19]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank0)

排名前5领涨板块成分股列表。
板块：昨日涨停_含一字:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态   市净率
0  688262  国芯科技  20.01  NaN  0.0  -66.25  5.00
1  688691  灿芯股份  19.99  NaN  0.0   85.61  6.80
2  300173  福能东方  19.97  NaN  0.0  572.26  5.68
3  833580  科创新材  19.05  NaN  0.0  161.29  5.11
4  300379   东方通  15.00  NaN  0.0  -52.50  3.67


板块：昨日涨停:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态   市净率
0  688262  国芯科技  20.01  NaN  0.0  -66.25  5.00
1  688691  灿芯股份  19.99  NaN  0.0   85.61  6.80
2  300173  福能东方  19.97  NaN  0.0  572.26  5.68
3  833580  科创新材  19.05  NaN  0.0  161.29  5.11
4  300379   东方通  15.00  NaN  0.0  -52.50  3.67


板块：昨日连板_含一字:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态    市净率
0     557  西部创业  10.06  NaN  0.0   33.62   1.65
1  600841  动力新科  10.04  NaN  0.0   -6.05   2.39
2  603366  日出东方  10.04  NaN  0.0   71.55   1.61
3     826  启迪环境  10.03  NaN  0.0   -3.06   1.56
4    2547  春兴精工  10.02  NaN  0.0  -25.23  89.67


板块：高带宽内存:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态    市净率
0  688262  国芯科技  20.01  NaN  0.0  

#### 2.2.3 开盘后15分钟概念板块排名

In [20]:
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 52)
rank1=recordRank(basename,start_time,end_time)
rank1.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,毛发医疗,BK0996,3.05,0.000062,0.76,1.000000,国际医学,10.07
1,2,昨日连板_含一字,BK1051,2.77,0.000383,3.27,0.746032,大港股份,10.00
2,3,SPD概念,BK1147,2.56,0.000048,1.13,1.000000,华人健康,4.82
3,4,3D摄像头,BK0901,2.53,0.000367,2.36,0.727273,晶方科技,9.34
4,5,昨日涨停_含一字,BK1050,2.50,0.001715,3.06,0.702830,粤桂股份,10.04
5,6,锂矿概念,BK1173,2.38,0.001601,0.88,0.756757,威领股份,9.97
6,7,屏下摄像,BK0947,2.22,0.000354,1.03,0.666667,维信诺,10.04
7,8,昨日连板,BK0816,2.20,0.000227,3.58,0.743590,大港股份,10.00
8,9,昨日涨停,BK0815,2.18,0.001512,3.35,0.693122,粤桂股份,10.04
9,10,ST股,BK0511,2.00,0.000268,0.71,0.773109,*ST中润,4.95


排名前5概念板块领涨成分股

In [12]:
top5_concept_stocks=top5sectorStocks(basename,start_time,end_time,rank1)

排名前5领涨板块成分股列表。
No record stored and not available from source.
No record stored and not available from source.
No record stored and not available from source.
No record stored and not available from source.
No record stored and not available from source.


#### 2.2.3 集合竞价行业板块涨跌幅

In [11]:
#当天集合竞价到开盘后30分钟涨跌幅排名
basename='industryRank'
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
rank3=recordRank(basename,start_time,end_time)
rank3.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,半导体,BK1036,1.89,0.045593,0.0,0.840000,国芯科技,20.01
1,2,船舶制造,BK0729,1.88,0.004153,0.0,1.000000,中国动力,10.02
2,3,电子化学品,BK1039,1.83,0.002984,0.0,0.913043,中巨芯-U,7.33
3,4,航天航空,BK0480,1.24,0.011011,0.0,0.875000,航天宏图,8.28
4,5,汽车服务,BK1016,1.06,0.000691,0.0,0.800000,ST东时,4.84
5,6,汽车整车,BK1029,0.72,0.021158,0.0,0.541667,动力新科,10.04
6,7,通信设备,BK0448,0.67,0.015660,0.0,0.592233,长江通信,10.02
7,8,计算机设备,BK0735,0.50,0.009903,0.0,0.511111,合众思壮,10.05
8,9,橡胶制品,BK1018,0.40,0.002831,0.0,0.576923,青岛双星,9.96
9,10,电子元件,BK0459,0.40,0.018731,0.0,0.547826,弘信电子,19.95


行业板块中成分股

In [13]:
top5_industry_stocks=top5sectorStocks(basename,start_time,end_time,rank3)

排名前5领涨板块成分股列表。
板块：半导体:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态     市净率
0  688262  国芯科技  20.01  NaN  0.0  -66.25    5.00
1  688691  灿芯股份  15.42  NaN  0.0   82.35    6.54
2  688362  甬矽电子  11.55  NaN  0.0  202.27    4.61
3    2077  大港股份  10.02  NaN  0.0  196.31    3.14
4     670   盈方微   9.96  NaN  0.0 -142.50  372.83


板块：船舶制造:
       代码    名称    涨跌幅  成交额  换手率  市盈率-动态   市净率
0  600482  中国动力  10.02  NaN  0.0   58.05  1.50
1  300008  天海防务   2.36  NaN  0.0  102.80  5.19
2  300123  亚光科技   1.70  NaN  0.0  -72.61  3.31
3  300589  江龙船艇   1.37  NaN  0.0  164.31  6.34
4  600685  中船防务   1.28  NaN  0.0  152.23  2.05


板块：电子化学品:
       代码     名称   涨跌幅  成交额  换手率    市盈率-动态    市净率
0  688549  中巨芯-U  7.25  NaN  0.0    446.76   5.81
1  300576   容大感光  4.60  NaN  0.0    131.37  13.74
2  300655   晶瑞电材  3.86  NaN  0.0  13837.20   4.75
3  300429   强力新材  3.56  NaN  0.0   -279.66   5.45
4  688548  XD广钢气  3.23  NaN  0.0     57.64   2.42


板块：航天航空:
       代码    名称   涨跌幅  成交额  换手率   市盈率-动态   市净率
0  688066  航天宏图  8.2

#### 2.2.4 开盘15分钟行业板块涨跌幅

In [23]:
basename='industryRank'
start_time = datetime.time(9, 31)
end_time = datetime.time(9, 51)
rank4=recordRank(basename,start_time,end_time)
rank4.head(20)

,排名,板块名称,板块代码,涨跌幅,市值比,换手率,上涨家数%,领涨股票,领涨股票-涨跌幅
0,1,能源金属,BK1015,4.66,0.003880,1.74,1.000000,威领股份,9.97
1,2,医药商业,BK1042,4.27,0.003478,0.91,1.000000,人民同泰,10.03
2,3,医疗服务,BK0727,2.53,0.007480,0.99,0.888889,国际医学,10.07
3,4,汽车整车,BK1029,2.16,0.022023,1.26,0.750000,东风股份,10.03
4,5,中药,BK1040,1.96,0.009787,1.77,0.944444,康惠制药,10.03
5,6,酿酒行业,BK0477,1.60,0.038877,0.47,0.948718,岩石股份,4.59
6,7,化学制药,BK0465,1.48,0.017352,0.62,0.888112,西点药业,10.52
7,8,小金属,BK1027,1.45,0.006910,0.95,0.675676,有研新材,10.01
8,9,美容护理,BK1035,1.42,0.003102,0.91,0.888889,润本股份,6.16
9,10,生物制品,BK1044,1.41,0.013166,0.59,0.853333,神州细胞-U,7.40


行业板块中成分股

In [24]:
top5_industry_stocks=top5sectorStocks(basename,start_time,end_time,rank4)

排名前5领涨板块成分股列表。
板块：能源金属:
       代码    名称    涨跌幅           成交额   换手率  市盈率-动态   市净率
0  300390  天华新能  11.15  9.091016e+08  4.64   20.24  2.09
1  002667  威领股份   9.97  1.417661e+08  5.11  -19.01  5.08
2  002738  中矿资源   8.10  1.136033e+09  3.90   41.04  2.51
3  002460  赣锋锂业   7.73  2.174438e+09  4.29 -100.78  1.96
4  002466  天齐锂业   4.43  1.588016e+09  2.53   -9.21  1.61


板块：医药商业:
       代码    名称    涨跌幅           成交额   换手率  市盈率-动态   市净率
0  300937   药易购  18.21  1.933751e+08  9.98  165.84  3.64
1  603716  塞力医疗  10.03  6.601823e+07  4.11  -22.78  1.75
2  600829  人民同泰  10.03  1.285216e+08  2.70   17.84  1.70
3  600998   九州通   6.80  1.572866e+08  0.54   13.31  1.22
4  600272  开开实业   6.54  7.933593e+07  3.71   72.05  5.37


板块：医疗服务:
       代码    名称    涨跌幅           成交额   换手率  市盈率-动态   市净率
0  000516  国际医学  10.07  1.779459e+08  1.23  -52.43  4.06
1  002219   新里程  10.00  2.185075e+08  2.13  154.46  4.70
2  301239  普瑞眼科   7.51  1.000922e+08  2.69  384.22  3.66
3  002622  皓宸医疗   6.84  7.633875e+07  2.87

### 2.2.5 集合竞价阶段涨停个股热点明细

In [25]:
#在指定时段获取涨停数据，若时间不在指定时段，则读取历史最后一次下载的记录。
def stockLimitUp(start_time,end_time):
    now = datetime.datetime.now()
    tStr = end_time.strftime("%Y%m%d%H%M")
    csvfile=f'limitUp{tStr}.txt'
    if now.time() < end_time and now.time() >= start_time:
        rank=ak.stock_zh_a_spot_em()
        rank.to_csv(csvfile)
    else:
        if os.path.exists(csvfile):
            rank=pd.read_csv(csvfile)
        else:
            print('No record stored and not available from source.')
            rank=pd.DataFrame()
    if not rank.empty:
        columns=['代码','名称','涨跌幅','量比','换手率','市盈率-动态','市净率','流通市值','涨速','5分钟涨跌','60日涨跌幅','年初至今涨跌幅']
        rank=rank[columns].sort_values(by='涨跌幅', ascending=False)
    return rank

In [26]:
start_time = datetime.time(9, 15)
end_time = datetime.time(9, 31)
openA = stockLimitUp(start_time,end_time)
openA.head(50)

,代码,名称,涨跌幅,量比,换手率,市盈率-动态,市净率,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,920111,N聚星,348.00,NaN,0.0,42.98,5.62,1.032878e+09,0.00,0.00,348.00,348.00
2,688262,国芯科技,20.01,NaN,0.0,-66.25,5.00,8.847816e+09,0.00,0.00,104.71,14.08
1,300493,润欣科技,20.01,NaN,0.0,178.79,8.25,8.652150e+09,0.00,0.00,122.27,103.90
3,300277,海联讯,19.97,NaN,0.0,801.72,9.92,4.821649e+09,0.00,0.00,127.80,67.09
4,300657,弘信电子,18.75,NaN,0.0,159.04,8.74,1.084570e+10,-1.04,-1.04,81.71,22.49
5,870436,大地电气,16.61,NaN,0.0,-52.62,3.66,7.372159e+08,-2.94,-6.25,127.27,50.82
6,688316,青云科技-U,16.49,NaN,0.0,-26.28,18.43,1.653434e+09,3.51,5.89,76.99,-11.18
7,300173,福能东方,16.04,NaN,0.0,553.54,5.50,4.996135e+09,-3.27,-3.27,97.67,38.49
8,688382,益方生物-U,10.54,NaN,0.0,-18.14,4.25,5.116356e+09,-1.54,-1.54,65.59,-15.46
9,2671,龙泉股份,10.10,NaN,0.0,28.46,1.80,2.949706e+09,0.00,0.00,47.51,-10.85


### 2.2.6 开盘15分钟涨停个股热点明细


In [27]:

start_time = datetime.time(9, 31)
end_time = datetime.time(9, 53)
open15 = stockLimitUp(start_time,end_time)
open15.head(50)

,代码,名称,涨跌幅,量比,换手率,市盈率-动态,市净率,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,430425,乐创技术,29.99,7.28,7.50,78.65,6.12,1.178450e+09,0.00,0.00,194.42,39.49
2,301209,联合化学,20.01,19.57,13.18,46.13,3.75,8.303050e+08,0.00,0.00,57.26,36.87
1,301000,肇民科技,20.01,8.40,2.66,36.20,4.49,4.783582e+09,0.00,0.00,61.50,41.99
3,300912,凯龙高科,20.00,19.12,6.24,-11.75,2.66,1.359285e+09,0.00,0.00,63.47,-12.05
4,301325,曼恩斯特,19.99,15.44,11.88,107.62,3.14,3.704103e+09,0.00,0.00,77.87,-0.02
5,300546,雄帝科技,19.99,9.44,7.84,298.59,3.98,3.066533e+09,0.00,0.00,107.84,46.16
6,688089,嘉必优,19.98,0.80,0.17,37.77,2.77,4.224559e+09,0.00,0.00,62.04,41.81
7,300277,海联讯,19.97,0.68,0.33,961.84,11.90,5.784641e+09,0.00,0.00,175.04,100.46
8,688678,福立旺,19.24,14.07,3.79,45.62,2.60,3.963574e+09,2.45,6.12,82.94,25.97
9,300937,药易购,16.74,12.84,10.47,163.78,3.59,1.992202e+09,-0.97,7.98,54.52,1.83


#### 2.2.7 板块动态评析

本轮行情启动股票热点追踪（需要从历史行情中组合分析，现有股票涨停池昨日和今日都不稳定）

In [28]:
from datetime import datetime, timedelta

def get_stock_zt_pool(start_date, end_date):
    current_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    
    concat_zt_df = pd.DataFrame()

    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        try:
            stock_zt_pool_strong_em_df = ak.stock_zt_pool_previous_em(date=date_str)
            stock_zt_pool_strong_em_df['日期'] = pd.to_datetime(date_str)
            concat_zt_df = pd.concat([concat_zt_df, stock_zt_pool_strong_em_df], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
        
        current_date += timedelta(days=1)
    
    return concat_zt_df

In [29]:
syclestart='20240925'
#ak.stock_zt_pool_strong_em(date=syclestart)

In [30]:
#本轮行情启动日
syclestart='20240924'
result_df = get_stock_zt_pool(syclestart, lastday)
result_df

NameError: name 'lastday' is not defined

## 4.个股市场


### 4.1 跟进股票列表
#### 4.1.1 历史K线

In [31]:
def plotDcStockSingle(df,days):
    OCHLVDict={'日期':'Date','开盘':'Open','收盘':'Close','最高':'High','最低':'Low','成交量':'Volume'}
    print(f'{days}K线图')
    plotStock(df.tail(days),OCHLVDict)

In [32]:
#根据代码获取股票信息
#symbol:若是字符串，则返回单个股信息；若是列表，则遍历元素，返回相应列表。
def getStockInfo(symbol):
    if type(symbol).__name__=='str':
        stocknamedf=ak.stock_individual_info_em(symbol=symbol)
        stockname=stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1]
        stockIPO=stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7]
        stockIndustry=stocknamedf.loc[stocknamedf['item']=='行业','value'][6]
        stockTAmount=stocknamedf.loc[stocknamedf['item']=='总股本','value'][2]
        stockFAmount=stocknamedf.loc[stocknamedf['item']=='流通股','value'][3]
        stockTValue=stocknamedf.loc[stocknamedf['item']=='总市值','value'][4]
        stockFValue=stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5]
    else:
        stockname,stockIPO,stockIndustry,stockTAmount,stockFAmount,stockTValue,stockFValue=[],[],[],[],[],[],[]
        for sym in symbol:
            stocknamedf=ak.stock_individual_info_em(symbol=sym)
            stockname.append(stocknamedf.loc[stocknamedf['item']=='股票简称','value'][1])
            stockIPO.append(stocknamedf.loc[stocknamedf['item']=='上市时间','value'][7])
            stockIndustry.append(stocknamedf.loc[stocknamedf['item']=='行业','value'][6])
            stockTAmount.append(stocknamedf.loc[stocknamedf['item']=='总股本','value'][2])
            stockFAmount.append(stocknamedf.loc[stocknamedf['item']=='流通股','value'][3])
            stockTValue.append(stocknamedf.loc[stocknamedf['item']=='总市值','value'][4])
            stockFValue.append(stocknamedf.loc[stocknamedf['item']=='流通市值','value'][5])
            
    return {'名称':stockname,'IPO':stockIPO,'行业':stockIndustry,'总股本':stockTAmount,'流通股':stockFAmount,'总市值':stockTValue,'流通市值':stockFValue}

#画指定周期的股票烛线图
def plotDcStocks(stocksymb,periods):
    stockDict={}
    for p in periods:
        stockdf = ak.stock_zh_a_hist(symbol=stocksymb,period=p)
        stockname=getStockInfo(stocksymb)['名称']
        stockDict[p]=stockdf
        print(f'{stockname}-周期：{p}\n{stockdf}')
        plotDcStockSingle(stockdf,100)
    stockDict['name']=stockname
    return stockDict

In [33]:
stocksymb='300339'
periods=['daily','weekly','monthly']
stockDict=plotDcStocks(stocksymb,periods)

润和软件-周期：daily
              日期    股票代码     开盘     收盘     最高     最低      成交量           成交额  \
0     2012-07-18  300339  28.00  23.90  28.00  23.21   150973  3.772116e+08   
1     2012-07-19  300339  24.40  24.07  25.30  23.71    93127  2.276391e+08   
2     2012-07-20  300339  24.00  24.19  24.78  23.92    61087  1.484579e+08   
3     2012-07-23  300339  23.51  23.20  23.59  21.78    60498  1.371628e+08   
4     2012-07-24  300339  23.60  25.52  25.52  23.50    81850  2.034769e+08   
...          ...     ...    ...    ...    ...    ...      ...           ...   
2837  2024-11-06  300339  68.02  67.30  71.23  66.66  1513340  1.036907e+10   
2838  2024-11-07  300339  66.00  68.79  69.95  64.50  1408939  9.423211e+09   
2839  2024-11-08  300339  69.78  71.80  75.50  69.78  1774061  1.297837e+10   
2840  2024-11-11  300339  70.60  75.49  78.00  68.96  1723875  1.274192e+10   
2841  2024-11-12  300339  75.20  73.77  75.49  73.37   269565  2.002645e+09   

         振幅    涨跌幅   涨跌额    换手率  
0  

NameError: name 'plotStock' is not defined

### 4.1.2 盘口信息汇总
#### 4.1.2.1 成交和挂单明细

获取腾讯当日历史分笔数据

In [ ]:
# 根据code代码返回“交易所+代码” 信息，或者反过来
def transferExSym(code):
    if len(code)>6:
        rescode=code[-6:]
    else:
        source={}
        source['sh']=ak.stock_sh_a_spot_em()['代码'].to_list()
        source['sz']=ak.stock_sz_a_spot_em()['代码'].to_list()
        source['bj']=ak.stock_bj_a_spot_em()['代码'].to_list()
        for k in source.keys():
            if code in source[k]:
                rescode=k+code
                break
    return rescode

In [34]:
import numpy as np
import matplotlib.pyplot as plt

#区分北交所和沪深股市，刻画市场深度。按照涨跌停板，作为极值，按照价格涨幅的measure度量区间
def priceMarketDepth(df,measure=0.01):

    open_price = df.loc[df['成交时间'] == '09:30:00', '成交价格'].iloc[0]

    # 设置涨跌停板系数
    if stocksymb == 'bj':
        limit_factor = 0.2
    else:
        limit_factor = 0.1

    # 计算最高价和最低价
    high_price = open_price * (1 + limit_factor)
    low_price = open_price * (1 - limit_factor)

    # 定义价格区间，每1%一档
    price_bins = np.round(np.arange(low_price, high_price, (high_price - low_price) * measure),2)

    # 将价格按照区间划分
    df['价格区间'] = pd.cut(df['成交价格'], bins=price_bins, right=False)

    # 按照价格区间汇总成交量
    volume_summary = df.groupby('价格区间',observed=False)['成交量'].sum().reset_index()

    # 新建 df['prop'] 列
    df['prop'] = df['性质'].apply(lambda x: -1 if x == '卖盘' else (1 if x == '买盘' else 0))
    
    # 按照价格区间汇总成交量和 prop 之和
    summary = df.groupby('价格区间',observed=False).agg({
        '成交量': 'sum',
        'prop': 'sum'
    }).reset_index()
    
    # 提取价格区间的中点和边界
    summary['价格'] = summary['价格区间'].apply(lambda x: round(x.mid,2))
    summary['起始价格'] = summary['价格区间'].apply(lambda x: round(x.left,2))
    summary['终止价格'] = summary['价格区间'].apply(lambda x: round(x.right,2))

    # 分离买盘和卖盘数据
    summary['买盘深度'] = summary['prop'].apply(lambda x: x if x > 0 else 0)
    summary['卖盘深度'] = -summary['prop'].apply(lambda x: -x if x < 0 else 0)

    # 绘制市场深度水平条形图
    fig, ax = plt.subplots(figsize=(14, 21))

    # 绘制买卖盘深度图
    ax.barh(summary['价格'], summary['买盘深度'], color='green', label='buy')
    ax.barh(summary['价格'], summary['卖盘深度'], color='red', label='sell')
    
    # 在图上标注区间价格的起始和终止价格
    for index, row in summary.iterrows():
        ax.text(row['买盘深度'] + max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["起始价格"]:.2f}', va='center', ha='left', fontsize=8, color='black')
        ax.text(-row['卖盘深度'] - max(summary['买盘深度'].max(), summary['卖盘深度'].max())*measure, row['价格'], f'{row["终止价格"]:.2f}', va='center', ha='right', fontsize=8, color='black')

    # 设置图表标题和标签
    ax.set_title('Market Depth')
    ax.set_xlabel('Depth(Amount)')
    ax.set_ylabel('Price')
    ax.legend()

    
    # 显示图表
    plt.show()
    
    return summary

In [35]:
from rich.console import Console
from rich.table import Table

#美观显示DataFrame表格
class DataFramePretty(object):
    def __init__(self, df: pd.DataFrame, min_max_cols=[]) -> None:
        self.data = df
        self.min_max_cols = min_max_cols

    def __set_min_max_style(self, df: pd.DataFrame):
        imax = self.data.idxmax()
        imin = self.data.idxmin()

        max_tmpl = "[b on red3]{}[/b on red3]"
        min_tmpl = "[i on green3]{}[/i on green3]"

        for col in self.min_max_cols:
            col_idx = imax.index.tolist().index(col)
            val = df.iloc[imax[col], col_idx]
            df.iloc[imax[col], col_idx] = max_tmpl.format(val)

            col_idx = imin.index.tolist().index(col)
            val = df.iloc[imin[col], col_idx]
            df.iloc[imin[col], col_idx] = min_tmpl.format(val)

    def show(self):
        table = Table(
            title="DataFrame",
            title_style="i on dark_cyan",
            header_style="bold cyan",
        )

        # self.data是原始数据
        # df 是用来显示的数据
        df = self.data.copy()
        for col in df.columns:
            df[col] = df[col].astype("str")
            table.add_column(col)

        self.__set_min_max_style(df)

        for idx in range(len(df)):
            table.add_row(*df.iloc[idx].tolist())

        console = Console()
        console.print(table)

In [ ]:
#剔除中性单以后，列出活跃买卖市场深度
def plotActiveMarketDepth(stocksymbol):
    
    print(f'市场深度：{stocksymb}')
    #一次性获得历史分笔数据（当天即当天之前的交易时间；隔天即昨天）
    yesterdayTicks=ak.stock_zh_a_tick_tx_js(symbol=transferExSym(stocksymb))
    
    md=priceMarketDepth(yesterdayTicks)
    md = md[(md['买盘深度'] != 0) | (md['卖盘深度'] != 0)]
    mdp=DataFramePretty(md)
    mdp.show()
    return  md

剔除中性盘以后，主动买卖盘的市场深度

In [ ]:
plotActiveMarketDepth(stocksymb)

### 4.1.3 个股追踪
追踪股票列表

In [ ]:
focusStocks=['300576','600839','600157','000717','002607','600127','600605','600217','300727','301421','002843','600968','300159','601688']
getInfo=getStockInfo(focusStocks)
Analist=pd.DataFrame({'代码':focusStocks,'名称':getInfo['名称'],'IPO':getInfo['IPO'],'行业':getInfo['行业'],'流通股':getInfo['流通股'],'总股本':getInfo['总股本']})
Analist['流通股']=Analist['流通股'].astype(int)
Analist['总股本']=Analist['总股本'].astype(int)
Analist['流通率']=Analist['流通股']/Analist['总股本']
Analist

时序数据库准备

In [ ]:
import dolphindb as ddb
from concurrent.futures import ThreadPoolExecutor
import dolphindb.settings as keys

s = ddb.session("localhost", 8848, "admin", "123456")
s.run("quote_tab = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, INT, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE,DOUBLE,DOUBLE])")
quote_tab = s.table(data="quote_tab")
dbPath="dfs://quotes"

if not s.existsDatabase(dbPath):
    db = s.database(dbPath=dbPath, partitionType=keys.LIST, partitions=focusStocks,engine='TSDB')
    db.createTable(quote_tab, "quotes", ["sym"])
    
qt = s.table(dbPath=dbPath, data="quotes")


In [ ]:
stockCharts,marketDepths={},{}

for f in focusStocks:
    print('='*75)
    stockCharts[f]=plotDcStocks(f,periods)
    stockname=stockCharts[f]['name']
    print(f'{stockname}（{f}）的市场深度：')
    marketDepths[f]=plotActiveMarketDepth(f)
    

In [ ]:
stock_bid_ask_em_df = ak.stock_bid_ask_em(symbol="600839")
stock_bid_ask_em_df

In [ ]:
import akshare as ak
import dolphindb as ddb
import pandas as pd
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# 初始化 DolphinDB 连接
s = ddb.session()
s.connect("localhost", 8848, "admin", "123456")  # 替换为实际的主机、端口、用户名和密码

# 创建 DolphinDB 表
s.run("""
if (existsDatabase("dfs://Quote")) {
    dropDatabase("dfs://Quote")
}
t = table(1:0, `timestamp`sym`sell_5`sell_5_vol`sell_4`sell_4_vol`sell_3`sell_3_vol`sell_2`sell_2_vol`sell_1`sell_1_vol`buy_1`buy_1_vol`buy_2`buy_2_vol`buy_3`buy_3_vol`buy_4`buy_4_vol`buy_5`buy_5_vol`最新`均价`涨幅`涨跌`总手`金额`换手`量比`最高`最低`今开`昨收`涨停`跌停`外盘`内盘, [TIMESTAMP, SYMBOL, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE])
database("dfs://Quote", VALUE, t)
""")

def is_within_trading_hours(current_time):
    # 定义交易时间段
    morning_start = current_time.replace(hour=9, minute=15, second=0, microsecond=0)
    morning_end = current_time.replace(hour=11, minute=30, second=0, microsecond=0)
    afternoon_start = current_time.replace(hour=13, minute=0, second=0, microsecond=0)
    afternoon_end = current_time.replace(hour=15, minute=0, microsecond=0)
    
    return (morning_start <= current_time <= morning_end) or (afternoon_start <= current_time <= afternoon_end)

def fetch_and_store(symbol):
    # 获取当前时间
    current_time = datetime.now()

    if is_within_trading_hours(current_time):fgggttryy
        # 获取股票数据
        df = ak.stock_bid_ask_em(symbol=symbol)

        # 转换为字典
        data_dict = df.set_index('item').to_dict()['value']

        # 添加时间戳和股票代码到数据字典
        data_dict['timestamp'] = current_time
        data_dict['sym'] = symbol

        # 转换数据为 DataFrame
        transformed_df = pd.DataFrame([data_dict])

        # 将数据插入到 DolphinDB
        s.upload({"data": transformed_df})
        s.run("""
        append!(loadTable("dfs://Quote", "t"), data)
        """)]]
        ]

while True:
    
    # 使用多线程获取多个股票代码的数据
    with ThreadPoolExecutor(max_workers=len(focusStocks)) as executor:
        executor.map(fetch_and_store, focusStocks)

    # 每3秒检查一次
    time.sleep(3)
